In [2]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('Aviation_Data.csv')

# 1. Standardize missing values
df.replace(['UNK', '', ' ', 'N/A', 'NaN'], np.nan, inplace=True)

# 2. Clean date columns - convert to proper datetime format
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')
df['Publication.Date'] = pd.to_datetime(df['Publication.Date'], errors='coerce')

# 3. Clean numeric columns - handle missing values and convert to numeric
numeric_columns = [
    'Total.Fatal.Injuries', 'Total.Serious.Injuries', 
    'Total.Minor.Injuries', 'Total.Uninjured', 'Number.of.Engines',
    'Latitude', 'Longitude'
]

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(0)  # Fill missing with 0 for injury counts

# 4. Clean text columns - standardize capitalization and remove extra spaces
text_columns = [
    'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category',
    'Make', 'Model', 'Weather.Condition', 'Broad.phase.of.flight',
    'Purpose.of.flight', 'Country', 'Location'
]

for col in text_columns:
    df[col] = df[col].astype(str).str.strip().str.title()

# 5. Handle specific categorical variables
df['Injury.Severity'] = df['Injury.Severity'].replace({
    'Fatal(1)': 'Fatal', 'Fatal(2)': 'Fatal', 'Fatal(3)': 'Fatal',
    'Fatal(4)': 'Fatal', 'Fatal(5)': 'Fatal', 'Fatal(6)': 'Fatal',
    'Fatal(7)': 'Fatal', 'Fatal(8)': 'Fatal'
})

# 6. Extract year and month for easier analysis in Tableau
df['Event.Year'] = df['Event.Date'].dt.year
df['Event.Month'] = df['Event.Date'].dt.month

# 7. Clean airport codes and names
df['Airport.Code'] = df['Airport.Code'].astype(str).str.upper().str.strip()
df['Airport.Name'] = df['Airport.Name'].astype(str).str.title().str.strip()

# 8. Standardize weather conditions
weather_mapping = {
    'Imc': 'IMC', 'Vmc': 'VMC', 'Unk': 'Unknown'
}
df['Weather.Condition'] = df['Weather.Condition'].replace(weather_mapping)

# 9. Remove completely empty columns if any
df = df.dropna(axis=1, how='all')

# 10. Create a severity score for analysis
df['Severity.Score'] = (
    df['Total.Fatal.Injuries'] * 3 + 
    df['Total.Serious.Injuries'] * 2 + 
    df['Total.Minor.Injuries'] * 1
)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['Event.Date'].min()} to {df['Event.Date'].max()}")
print(f"Number of accidents: {len(df[df['Investigation.Type'] == 'Accident'])}")
print(f"Number of incidents: {len(df[df['Investigation.Type'] == 'Incident'])}")

# Save cleaned file
df.to_csv('Aviation_Data_Cleaned.csv', index=False)

<ipython-input-2-499891f188b9>:5: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Aviation_Data.csv')
c:\Users\kanja\anaconda3\envs\learn-env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '19-09-1996' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\kanja\anaconda3\envs\learn-env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26-02-2007' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\kanja\anaconda3\envs\learn-env\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16-04-1980' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  c

Dataset shape: (90348, 34)
Date range: 1948-10-24 00:00:00 to 2022-12-29 00:00:00
Number of accidents: 85015
Number of incidents: 3874
